In [ ]:
!pip install pytorch-lightning
!pip install torchmetrics
!pip install torchvision
!pip install torch torchvision torchaudio
!pip install matplotlib
!pip install numpy
!pip install tqdm
!pip install pandas
!pip install scikit-learn
!pip install seaborn

import torch
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision.datasets import CIFAR10,Imagenette
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as L
import torchmetrics
from torch.utils.data import DataLoader
from torchvision import transforms


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 62.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
class BaselineModel(L.LightningModule):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Linear(64 * 64, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
class BasicCNN(L.LightningModule):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(200704, 128)  # must match
        self.fc2 = nn.Linear(128, num_classes)
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
#BasicCNN with early dropjout
class ComplexCNN(L.LightningModule):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = None  # none for now
        self.fc2 = nn.Linear(128, num_classes)
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = x.to(self.device)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(self.dropout1(F.relu(self.conv2(x))))
        x = x.view(x.size(0), -1)  # Flatten

        # fc1 dynamic init. based on input shape every time
        if self.fc1 is None or self.fc1.in_features != x.shape[1]:
            self.fc1 = nn.Linear(x.shape[1], 128).to(self.device)

        x = self.dropout2(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def load_pretrained_weights(self, weights_path):
        pretrained_dict = torch.load(weights_path)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [ ]:
class ResNet18Model(L.LightningModule):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()
        self.model = models.resnet18(pretrained=pretrained)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.accuracy(y_hat, y)
        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
# class TransferLearningModel(L.LightningModule):
#     def __init__(self, num_classes=10, pretrained=True):
#         super().__init__()
#         self.model = models.resnet18(pretrained=pretrained)
#         self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
#         self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

#     def forward(self, x):
#         return self.model(x)

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         self.log("train_loss", loss)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         self.accuracy(y_hat, y)
#         self.log("val_accuracy", self.accuracy)
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         self.accuracy(y_hat, y)
#         self.log("test_accuracy", self.accuracy)
#         self.log("test_loss", loss)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         return optimizer


In [ ]:
# Data preparation
imagenette_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size (e.g., 224x224)
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Transform for transfer learning model with resize
# transfer_learning_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# Import shutil for removing directories
import shutil

# Remove the existing directory
# ONLY NEEDED WHEN RE-RUNNING.
#shutil.rmtree('/content/imagenette2/imagenette2')

imagenette_train_dataset = Imagenette(root='/content/imagenette2', split='train', download=True, transform=imagenette_transform)
imagenette_train_loader = DataLoader(imagenette_train_dataset, batch_size=64, shuffle=True)

imagenette_val_dataset = Imagenette(root='/content/imagenette2', split='val', download=False, transform=imagenette_transform)
imagenette_val_loader = DataLoader(imagenette_val_dataset, batch_size=64, shuffle=False)

# Imagenette Data for Transfer Learning (use transfer_learning_transform)
# imagenette_train_dataset_transfer = Imagenette(root='/content/imagenette2', split='train', download=False, transform=transfer_learning_transform)
# imagenette_train_loader_transfer = DataLoader(imagenette_train_dataset_transfer, batch_size=64, shuffle=True)

# imagenette_val_dataset_transfer = Imagenette(root='/content/imagenette2', split='val', download=False, transform=transfer_learning_transform)
# imagenette_val_loader_transfer = DataLoader(imagenette_val_dataset_transfer, batch_size=64, shuffle=False)

# CIFAR10 Data
cifar10_train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_train_loader = DataLoader(cifar10_train_dataset, batch_size=64, shuffle=True)

cifar10_val_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar10_val_loader = DataLoader(cifar10_val_dataset, batch_size=64, shuffle=False)

100%|██████████| 1.56G/1.56G [01:31<00:00, 16.9MB/s]


Extracting /content/imagenette2/imagenette2.tgz to /content/imagenette2


100%|██████████| 170M/170M [00:10<00:00, 16.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Training and evaluation for BasicCNN
basic_cnn_model = BasicCNN(num_classes=10)
trainer = L.Trainer(max_epochs=10, accelerator='auto', log_every_n_steps=10)
trainer.fit(basic_cnn_model, imagenette_train_loader, imagenette_val_loader)

# Test the BasicCNN model
basic_cnn_test_results = trainer.test(basic_cnn_model, imagenette_val_loader)
print("BasicCNN Test Results:", basic_cnn_test_results)

# Output for BasicCNN
print("\n--- BasicCNN Model ---")
print("Training and validation complete.")
print("Test Results:", basic_cnn_test_results)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | conv1    | Conv2d             | 896    | train
1 | conv2    | Conv2d             | 18.5 K | train
2 | pool     | MaxPool2d          | 0      | train
3 | fc1      | Linear             | 25.7 M | train
4 | fc2      | Linear             | 1.3 K  | train
5 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
25.7 M    Trainable params
0         Non-trainable params
25.7 M    Total params
102.844   Total estimated model params size (MB)
6         Modules in tr

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5747770667076111     │
│         test_loss         │     3.018395185470581     │
└───────────────────────────┴───────────────────────────┘

BasicCNN Test Results: [{'test_accuracy': 0.5747770667076111, 'test_loss': 3.018395185470581}]

--- BasicCNN Model ---
Training and validation complete.
Test Results: [{'test_accuracy': 0.5747770667076111, 'test_loss': 3.018395185470581}]


In [ ]:
# Training and evaluation for ResNet18Model
resnet18_model = ResNet18Model(num_classes=10)
trainer = L.Trainer(max_epochs=10, accelerator='auto', log_every_n_steps=10)
trainer.fit(resnet18_model, imagenette_train_loader, imagenette_val_loader)
resnet18_test_results = trainer.test(resnet18_model, imagenette_val_loader)

# Output for ResNet18
print("\n--- ResNet18 Model ---")
print("Training and validation complete.")
print("Test Results:", resnet18_test_results)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | ResNet             | 11.2 M | train
1 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8917197585105896     │
│         test_loss         │    0.4217335879802704     │
└───────────────────────────┴───────────────────────────┘


--- ResNet18 Model ---
Training and validation complete.
Test Results: [{'test_accuracy': 0.8917197585105896, 'test_loss': 0.4217335879802704}]


In [ ]:
# Training and evaluation for ComplexCNN (pretrained on Imagenette)
complex_cnn_pretrained_model = ComplexCNN(num_classes=10)
# Pre-train on Imagenette
print("\n--- ComplexCNN Model: Pre-training on Imagenette ---")
trainer = L.Trainer(max_epochs=10, accelerator='auto', log_every_n_steps=10)
trainer.fit(complex_cnn_pretrained_model, imagenette_train_loader, imagenette_val_loader)

# Save the pre-trained weights
torch.save(complex_cnn_pretrained_model.state_dict(), 'complex_cnn_pretrained_weights.pth')

# Load the pre-trained weights and fine-tune on CIFAR10
complex_cnn_pretrained_model.load_pretrained_weights('complex_cnn_pretrained_weights.pth')  # Load weights
trainer.fit(complex_cnn_pretrained_model, cifar10_train_loader, cifar10_val_loader)

# Test the ComplexCNN (pretrained) model
complex_cnn_pretrained_test_results = trainer.test(complex_cnn_pretrained_model, cifar10_val_loader)

# Output for ComplexCNN (pretrained)
print("\n--- ComplexCNN Model (Pretrained) ---")
print("Training and validation complete.")
print("Test Results:", complex_cnn_pretrained_test_results)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



--- ComplexCNN Model: Pre-training on Imagenette ---


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | conv1    | Conv2d             | 896    | train
1 | dropout1 | Dropout            | 0      | train
2 | conv2    | Conv2d             | 18.5 K | train
3 | dropout2 | Dropout            | 0      | train
4 | pool     | MaxPool2d          | 0      | train
5 | fc2      | Linear             | 1.3 K  | train
6 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
20.7 K    Trainable params
0         Non-trainable params
20.7 K    Total params
0.083     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.11180000007152557    │
│         test_loss         │    2.3533670902252197     │
└───────────────────────────┴───────────────────────────┘


--- ComplexCNN Model (Pretrained) ---
Training and validation complete.
Test Results: [{'test_accuracy': 0.11180000007152557, 'test_loss': 2.3533670902252197}]


In [ ]:
# Training and evaluation for ComplexCNN (from scratch on CIFAR10)
complex_cnn_scratch_model = ComplexCNN(num_classes=10)  # Create a new instance
trainer = L.Trainer(max_epochs=10, accelerator='auto', log_every_n_steps=10)
trainer.fit(complex_cnn_scratch_model, cifar10_train_loader, cifar10_val_loader)

# Test the ComplexCNN (from scratch) model
complex_cnn_scratch_test_results = trainer.test(complex_cnn_scratch_model, cifar10_val_loader)

# Output for ComplexCNN (from scratch)
print("\n--- ComplexCNN Model (From Scratch) ---")
print("Training and validation complete.")
print("Test Results:", complex_cnn_scratch_test_results)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | conv1    | Conv2d             | 896    | train
1 | dropout1 | Dropout            | 0      | train
2 | conv2    | Conv2d             | 18.5 K | train
3 | dropout2 | Dropout            | 0      | train
4 | pool     | MaxPool2d          | 0      | train
5 | fc2      | Linear             | 1.3 K  | train
6 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
20.7 K    Trainable params
0         Non-trainable params
20.7 K    Total params
0.083     Total estim

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5694000124931335     │
│         test_loss         │    1.3909417390823364     │
└───────────────────────────┴───────────────────────────┘


--- ComplexCNN Model (From Scratch) ---
Training and validation complete.
Test Results: [{'test_accuracy': 0.5694000124931335, 'test_loss': 1.3909417390823364}]


In [ ]:
# # Training and evaluation for TransferLearningModel


# transfer_learning_model = TransferLearningModel(num_classes=10)
# # Pre-train on Imagenette
# print("\n--- Transfer Learning Model: Pre-training on Imagenette ---")
# trainer.fit(transfer_learning_model, imagenette_train_loader_transfer, imagenette_val_loader_transfer)

# # Fine-tune on CIFAR10
# trainer.fit(transfer_learning_model, cifar10_train_loader, cifar10_val_loader)

# # Test the Transfer Learning model
# transfer_learning_test_results = trainer.test(transfer_learning_model, cifar10_val_loader)

# # Output for Transfer Learning
# print("\n--- Transfer Learning Model ---")
# print("Training and validation complete.")
# print("Test Results:", transfer_learning_test_results)